### **Entrenamiento de alineación y razonamiento con RLHF**

Este cuaderno está pensado para ejecutarse **dentro de Docker** con un entorno que ya tenga instalados:

- `torch`
- `transformers`
- `datasets`
- `trl`
- (opcional) `accelerate` y soporte de GPU



#### **Preparación de entorno**

En esta sección:

- Elegimos dispositivo (CPU / GPU).
- Verificamos versiones de librerías clave.


In [1]:
import torch
import transformers
import datasets
import trl

print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)
print("TRL:", trl.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch: 2.2.0+cu121
Transformers: 4.42.3
Datasets: 2.20.0
TRL: 0.9.6


device(type='cpu')

#### **Entrenamiento de alineación (Alignment Training)**

**Entrenamiento de alineación** = conjunto de técnicas para que un modelo de lenguaje grande (LLM) sea:

- **Útil (helpful)**: responde a lo que el usuario realmente necesita.
- **Inofensivo (harmless)**: evita instrucciones dañinas, ilegales o poco éticas.
- **Honesto (honest)**: evita *alucinaciones* y reconoce sus límites.

Pipeline típico usual elemental:

1. **Pre-entrenamiento** (*pre-training*):  
   El modelo aprende estadística del lenguaje en grandes corpus, optimizando una tarea de modelado de lenguaje (predecir el siguiente token).

2. **Supervised Fine-Tuning (SFT)**:  
   Se ajusta el modelo con pares *(instrucción, respuesta deseada)*.  
   - Datos generados por humanos (anotadores) o por otros modelos filtrados.
   - Objetivo: que el modelo hable en "modo asistente".

3. **Modelado de recompensas (Reward modeling)**:  
   Se entrena un modelo que asigna un **score escalar** a una respuesta, dado un prompt.  
   - Usa datos de **preferencias humanas**: pares *(chosen, rejected)*.
   - El modelo aprende a puntuar más alto las respuestas alineadas.

4. **RLHF (Reinforcement Learning from Human Feedback)**:  
   El reward model actúa como **juez**. Con algoritmos como PPO:
   - La *política (policy)* (modelo generador) produce respuestas.
   - El modelo de recompensa las puntúa.
   - La política  se actualiza para maximizar la recompensa (con regularización por KL para no alejarse mucho del modelo base).

En este cuaderno trabajaremos con un subconjunto del dataset **`Anthropic/hh-rlhf`** para ilustrar el *reward modeling** y un esqueleto de RLHF.


### **Tipos de retroalimentación humana y dataset `Anthropic/hh-rlhf`**

##### **Tipos de human feedback**

En la práctica, la retroalimentación humana puede tomar varias formas:

1. **Etiquetas directas (supervisión estándar)**  
   - Por ejemplo: *"esta respuesta es correcta/incorrecta"*, *"esta respuesta es tóxica"*, etc.

2. **Preferencias pareadas (pairwise preferences)**  
   - Dado un mismo prompt, el humano ve dos respuestas y elige cuál prefiere.
   - Es justo el caso de `hh-rlhf`: `chosen` vs `rejected`.

3. **Recompensas escalares**  
   - El humano da un score (por ejemplo, entre 1 y 5).

4. **Feedback estructurado**  
   - Explicaciones, correcciones paso a paso, anotaciones de razonamiento, etc.

En RLHF nos interesan especialmente las **preferencias pareadas**, porque permiten entrenar un modelo de recompensas robusto con relativamente pocos datos.

#### **`Anthropic/hh-rlhf`**

`Anthropic/hh-rlhf` contiene diálogos tipo chat con dos variantes para cada ejemplo:

- `chosen`: respuesta preferida (más alineada y segura).
- `rejected`: respuesta descartada (menos segura, menos útil).

En un escenario de seguridad, por ejemplo, `chosen` suele ser la respuesta que:

- **Rechaza** ayudar a hacer daño (ejemplo, hackear correos, fabricar explosivos).
- Ofrece alternativas seguras o educativas.

Mientras que `rejected` a menudo:

- Da detalles de cómo ejecutar la acción dañina,
- O al menos no la bloquea de forma clara.

Nuestro reward model aprenderá a asignar **score mayor** a `chosen` que a `rejected`.


In [2]:
from datasets import load_dataset

# Cargamos un subconjunto pequeño para exploración
raw_ds = load_dataset("Anthropic/hh-rlhf", split="train[:200]")
raw_ds

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 200
})

In [3]:
example = raw_ds[0]
print(example.keys())

print("\n Escogida (respuesta preferida) \n")
print(example["chosen"][:600])

print("\n Rechazada (respuesta no deseada)\n")
print(example["rejected"][:600])

dict_keys(['chosen', 'rejected'])

 Escogida (respuesta preferida) 



Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: I haven't even thought about it.

 Rechazada (respuesta no deseada)



Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow,

#### **Preparar los datos para `RewardTrainer`**

El `RewardTrainer` de TRL espera un dataset con, al menos, columnas como:

- `input_ids_chosen`, `attention_mask_chosen`
- `input_ids_rejected`, `attention_mask_rejected`

Pasos:

1. Elegimos un modelo base para el modelo de recompensas (aquí, **`gpt2-medium`**, mayor que `sshleifer/tiny-gpt2`).  
2. Tokenizamos `chosen` y `rejected` con la misma longitud máxima.  
3. Filtramos ejemplos demasiado largos.

Esto nos dará un dataset listo para el entrenamiento del modelo de recompensa.


In [4]:
from transformers import AutoTokenizer

reward_model_name = "gpt2-medium"  
tokenizer = AutoTokenizer.from_pretrained(reward_model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

max_length = 512

def build_pair(example):
    chosen_text = example["chosen"]
    rejected_text = example["rejected"]

    chosen_tokens = tokenizer(
        chosen_text,
        truncation=True,
        max_length=max_length,
        padding=False,
    )
    rejected_tokens = tokenizer(
        rejected_text,
        truncation=True,
        max_length=max_length,
        padding=False,
    )

    return {
        "input_ids_chosen": chosen_tokens["input_ids"],
        "attention_mask_chosen": chosen_tokens["attention_mask"],
        "input_ids_rejected": rejected_tokens["input_ids"],
        "attention_mask_rejected": rejected_tokens["attention_mask"],
    }

processed_ds = raw_ds.map(build_pair, remove_columns=raw_ds.column_names)

processed_ds = processed_ds.filter(
    lambda x: len(x["input_ids_chosen"]) <= max_length
              and len(x["input_ids_rejected"]) <= max_length
)

processed_ds

Dataset({
    features: ['input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
    num_rows: 200
})

#### **Entrenamiento de un reward model con `RewardTrainer`**

Usaremos:

- `AutoModelForSequenceClassification` con `num_labels = 1`.
- `RewardTrainer` de TRL con una configuración simple.

La pérdida que se optimiza es de tipo **Bradley-Terry**:
el modelo recibe scores $r_{chosen}, r_{rejected}$ y aprende que:

- $r_{chosen} > r_{rejected}$

De forma más formal, se maximiza algo como:

$$
\log \sigma(r_{chosen} - r_{rejected})
$$

donde $\sigma$ es la sigmoide logística.


In [5]:
from transformers import AutoModelForSequenceClassification
from trl import RewardTrainer, RewardConfig

reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_name,
    num_labels=1,
)
reward_model.to(device)
reward_model.config.pad_token_id = tokenizer.pad_token_id
reward_model.config.use_cache = False

reward_train_config = RewardConfig(
    output_dir="./reward-model-hh-rlhf-demo",
    per_device_train_batch_size=2,
    num_train_epochs=1,  # demo: 1 epoch
    learning_rate=1e-5,
    logging_steps=10,
    gradient_accumulation_steps=1,
    remove_unused_columns=False,
    max_length=max_length,
)

reward_trainer = RewardTrainer(
    model=reward_model,
    args=reward_train_config,
    tokenizer=tokenizer,
    train_dataset=processed_ds,
)

print("RewardTrainer inicializado.")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RewardTrainer inicializado.


In [6]:
# El entrenamiento completo puede ser costoso.
# Para un entorno de demo puedes reducir el número de ejemplos o epochs.
#
# Descomenta para entrenar de verdad:
#
# reward_trainer.train()
# reward_trainer.save_model()
# tokenizer.save_pretrained("./reward-model-hh-rlhf-demo")

#### **Esqueleto de RLHF con `PPOTrainer`**

El ciclo básico de PPO (muy simplificado) es:

1. Tomar un batch de **prompts** (queries).
2. La política(modelo generador) produce respuestas.
3. El modelo de recompensa asigna una recompensa a cada respuesta.
4. PPO ajusta la política para maximizar la recompensa, penalizando desviarse
   demasiado del modelo de referencia (regularización por KL).

Aquí mostraremos un **esqueleto** donde la recompensa es *dummy* (por longitud)
para ilustrar la API de `PPOTrainer`. En un pipeline real (no incluido aquí), se conectaría con el modelo de recompensas entrenado en la sección anterior.


In [7]:
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from transformers import AutoTokenizer

policy_model_name = "gpt2"  # policy para PPO (más grande que tiny)
ppo_tokenizer = AutoTokenizer.from_pretrained(policy_model_name)
if ppo_tokenizer.pad_token is None:
    ppo_tokenizer.pad_token = ppo_tokenizer.eos_token

policy_model = AutoModelForCausalLMWithValueHead.from_pretrained(policy_model_name)
policy_model.to(device)
ref_model = create_reference_model(policy_model)  # modelo de referencia fijo

ppo_config = PPOConfig(
    model_name=policy_model_name,
    batch_size=2,
    forward_batch_size=1,
    learning_rate=1e-5,
    mini_batch_size=1,
    log_with=None,
)

def extract_prompt(example):
    """
    Extrae una línea no vacía del campo 'chosen'.
    Si no encuentra nada, usa un prompt por defecto.
    """
    text = example["chosen"] or ""
    # nos quedamos solo con líneas que tengan contenido
    lines = [l for l in text.splitlines() if l.strip()]
    if lines:
        first_line = lines[0].strip()
    else:
        # fallback seguro (no debe quedar vacío)
        first_line = "Explain briefly what RLHF (Reinforcement Learning from Human Feedback) is."
    return {"query": first_line}

# Construir dataset con queries no vacías
ppo_ds = raw_ds.map(extract_prompt)

# Filtrar cualquier ejemplo que aun así tenga query vacía
ppo_ds = ppo_ds.filter(lambda ex: len(ex["query"].strip()) > 0)

# Usar un mini-dataset para la demo
ppo_ds = ppo_ds.select(range(min(8, len(ppo_ds))))

ppo_trainer = PPOTrainer(
    model=policy_model,
    ref_model=ref_model,
    config=ppo_config,
    tokenizer=ppo_tokenizer,
    dataset=ppo_ds,
)

print("PPOTrainer inicializado con queries limpias.")


/opt/conda/lib/python3.9/site-packages/trl/trainer/ppo_config.py:154: UserWarning: Note that using `forward_batch_size` is deprecated, use `mini_batch_size` instead. By setting it you overwrite `mini_batch_size` which affects both the batch size during forward passes and also the mini batch size for PPO optimization.
  warnings.warn(


PPOTrainer inicializado con queries limpias.


In [8]:
# Ejemplo de UN paso PPO con reward dummy basado en longitud de respuesta.

import torch

gen_kwargs = dict(
    max_new_tokens=32,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
)

# Tomamos un mini-batch desde el dataset interno del PPOTrainer
batch = ppo_trainer.dataset[: ppo_trainer.config.batch_size]
raw_queries = [q for q in batch["query"] if q.strip()]

if not raw_queries:
    raise ValueError("No hay queries válidas (todas vacías). Revisa el dataset o el filtro.")

# 1) Convertimos las queries (strings) a tensores de tokens
query_tensors = []
for q in raw_queries:
    enc = ppo_tokenizer(
        q,
        return_tensors="pt",
        truncation=True,
        max_length=128,
    )
    # enc["input_ids"]: shape (1, seq_len) -> lo dejamos en (seq_len,)
    query_tensors.append(enc["input_ids"].squeeze(0).to(device))

# 2) Generamos respuestas a partir de cada query_tensor
response_tensors = []
for q_tensor in query_tensors:
    # q_tensor: shape (seq_len,) -> expandimos a batch=1
    inputs = {"input_ids": q_tensor.unsqueeze(0).to(device)}
    outputs = policy_model.generate(
        **inputs,
        pad_token_id=ppo_tokenizer.eos_token_id,  # define padding y evita parte del ruido
        **gen_kwargs,
    )
    # Nos quedamos solo con la parte generada (después del prompt)
    resp = outputs[0, inputs["input_ids"].shape[-1]:]
    response_tensors.append(resp)

# 3) Reward dummy = longitud de la respuesta
#    IMPORTANTE: TRL quiere lista de tensores, no floats.
rewards = []
for resp in response_tensors:
    length = resp.shape[-1]
    rewards.append(torch.tensor(float(length)))  # tensor escalar

# 4) Paso PPO: queries, responses y rewards son listas de tensores
stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
stats


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'objective/kl': 0.0,
 'objective/kl_dist': array([0., 0.], dtype=float32),
 'objective/logprobs': array([[-5.0454917e+00, -5.5828285e+00, -2.9713676e+00, -3.1973491e+00,
         -1.0266676e+01, -5.1310158e+00, -3.9688346e-01, -3.0542512e+00,
         -4.7354078e+00, -7.1069807e-01, -1.0574325e+00, -9.3989745e-03,
         -4.8161550e+00, -3.1059086e+00, -2.6087179e+00, -4.6385756e+00,
         -4.1160727e+00, -2.6365438e+00, -1.0174114e+00, -2.1193166e-04,
         -4.1726003e+00, -3.3575612e-01, -8.4118974e-01, -5.4087420e+00,
         -2.6615305e+00, -1.2992363e+00, -1.6907987e-01, -1.9953644e-04,
         -1.3743496e+00, -1.9560654e-02, -1.9289745e-01, -4.5840139e+00,
         -3.8360395e+00, -2.1989255e+00, -1.4569078e-01, -1.2038434e-01,
         -2.3171601e-04, -3.2421359e-01, -1.9321332e-02, -1.1621184e-01,
         -4.8667555e+00, -2.5975082e+00],
        [-5.0454917e+00, -5.5828247e+00, -4.6213140e+00, -3.2080401e-02,
         -1.0034083e+01, -4.2052121e+00, -9.1327219e+00, 

> En un pipeline real de RLHF, sustituirías el `reward dummy` por el output
> del reward model entrenado con `RewardTrainer` (sección 5).


### **Alucinaciones (Hallucinations) y técnicas de mitigación**

#### **¿Qué es una alucinación?**

Una **alucinación** es una salida del modelo que:

- Es gramaticalmente correcta y plausible,
- Pero es **falsa**, inventada o no respaldada por el contexto o el conocimiento.

Ejemplos típicos:

- Inventar referencias bibliográficas / papers inexistentes.
- Describir APIs, funciones o rutas de archivos que no existen.
- Dar instrucciones técnicas incorrectas con aparente seguridad.

#### **Auto-consistencia (*Self-Consistency*)**

Estrategia:

1. Generar múltiples respuestas para la misma pregunta (sampling).
2. Combinar las respuestas (voto mayoritario, agregación estadística).

Intuición:

- Si la respuesta correcta es relativamente "estable" en la distribución del modelo,
  tenderá a aparecer con más frecuencia que las alucinaciones.


In [9]:
from collections import Counter

question = "¿Cuál es la capital de Francia?"

def generate_answers(model, tokenizer, prompt, n_samples=5):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=16,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
    )
    gen = outputs[:, inputs["input_ids"].shape[-1]:]
    return tokenizer.batch_decode(gen, skip_special_tokens=True)

samples = generate_answers(policy_model, ppo_tokenizer, question, n_samples=5)
for i, s in enumerate(samples, 1):
    print(f"[{i}] {s}")

normalized = [s.strip().lower() for s in samples]
counts = Counter(normalized)

print("\nAuto-consistencia (conteo):")
for ans, c in counts.most_common():
    print(c, "->", ans)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[1] 

Yes, but not in the sense of the capital of the nation of

Auto-consistencia (conteo):
1 -> yes, but not in the sense of the capital of the nation of


#### **Cadena de acciones (*Chain-of-Actions*)**

Similar a **Chain-of-Thought**, pero centrado en acciones concretas:

- "Primero valida la entrada."  
- "Luego comprueba permisos."  
- "Después consulta la base de datos."  
- "Finalmente formatea la respuesta."  

Ventajas:

- El razonamiento se vuelve **auditado** y más interpretable.
- Se pueden insertar **guardrails de seguridad** entre pasos
  (por ejemplo, revisar si la acción solicitada es permitida antes de seguir).

#### **Recitación (*Recitation*)**

Técnica para separar:

1. Recuperación de conocimiento.
2. Respuesta final.

Prompt típico:

> "Antes de responder, enumera los hechos relevantes sobre X.  
> Después, usando solo esos hechos, responde la pregunta."

Esto obliga al modelo a explicitar su "mini base de conocimientos" antes de responder.


#### **Métodos de muestreo para abordar alucinaciones**

Los hiperparámetros de generación influyen mucho en la probabilidad de alucinar:

- **temperature**  
  - Alta -> más creatividad, más riesgo de inventar.  
  - Baja -> más conservadora, pero a veces repetitiva.

- **top_p (nucleus sampling)** y **top_k**  
  - Restringen el conjunto de tokens candidatos en cada paso.
  - Valores moderados (ejemplo: `top_p=0.9`) suelen dar buen equilibrio.

- **max_new_tokens**  
  - Limita longitud de la respuesta para evitar divagaciones.

En contextos sensibles (seguridad, medicina, etc.) suele optarse por:

- `temperature` baja (0.2–0.7),
- `top_p` moderado (0.8–0.9),
- y controles adicionales (verificación externa, herramientas, etc.).


In [10]:
prompt = "Resume brevemente qué es RLHF (Reinforcement Learning from Human Feedback)."

for temp in [0.3, 0.7, 1.2]:
    inputs = ppo_tokenizer(prompt, return_tensors="pt").to(device)
    outputs = policy_model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=True,
        top_p=0.9,
        temperature=temp,
    )
    gen = outputs[:, inputs["input_ids"].shape[-1]:]
    text = ppo_tokenizer.batch_decode(gen, skip_special_tokens=True)[0]
    print(f"\n temperature{temp}\n{text}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 temperature0.3


"I am not sure how to say this, but I think it is a good thing. I am not sure how to say this, but I think it is a good thing. I am not sure how to say this, but I am not sure how to say this, but I am not sure how



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 temperature0.7
 The following are the results of the experiments.

The experiment

We first use the SRS to detect a training stimulus, and then to test the strength of the response. The training stimulus is a single word. The training stimulus is a single. The training stimulus is a single word, as it's a


 temperature1.2


I want to see the most of that, that's what we need to do and what we do as our competitors,
I want to know all of the rules and what are those, and to say, oh ok.
Well i did just give you a chance to just to try and I did a



#### **Decoding by Contrasting Layers (DoLa)**

**DoLa (Decoding by Contrasting Layers)** es una estrategia de decodificación que:

1. Obtiene logits (predicciones antes de softmax) de capas "tempranas" y capas "tardías" del Transformer.
2. Construye una distribución *contrastiva* (por ejemplo, `logits_mature - logits_premature`).
3. Aplica softmax sobre esa distribución para elegir el siguiente token.

Intuición:

- El conocimiento factual tiende a consolidarse en ciertas capas intermedias/tardías.
- Restar el ruido de capas tempranas ayuda a reducir algunas alucinaciones.

Implementación práctica:

- Hay que instrumentar el modelo con *hooks* para leer las activaciones.
- Algunas implementaciones añaden parámetros como `dola_layers="high"` a `generate`,
  pero **no es parte oficial de la API estándar de `transformers`**.

Ejemplo **conceptual** (no ejecutable con la API estándar sin una implementación específica): 

```python
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=False,
    dola_layers="high",  # requiere una implementación concreta de DoLa
)
```

En este cuaderno lo tratamos solo a nivel conceptual; la implementación real depende del repositorio de DoLa que utilices.


#### **Alucinaciones en contexto (*In-Context Hallucinations*)**

Suceden cuando:

- El contexto proporcionado realmente contiene la respuesta correcta,
- Pero el modelo aun así inventa detalles inconsistentes.

Ejemplo clásico:

- RAG mal configurado donde el modelo mezcla pasajes no relacionados y produce
  una conclusión que no está explícita en ningún documento.

#### **Alucinaciones por información irrelevante**

Si el prompt incluye muchos detalles irrelevantes, el modelo:

- Intentará "hacer sentido" de todo,
- Puede crear conexiones inexistentes (alucinaciones narrativas).

Mitigaciones típicas:

1. **Filtrado del contexto** (mejor retriever, rerankers, etc.).  
2. **Instrucciones claras**: "si no estás seguro, di que no lo sabes".  
3. **Controlar longitud y ruido** en el contexto.  
4. **Verificación con herramientas externas** (APIs, bases de datos, etc.).


### **Razonamiento (Reasoning): deductivo, inductivo y abductivo**

#### **Razonamiento deductivo**

- De reglas generales -> casos particulares.
- Si las premisas son verdaderas y la lógica es correcta, la conclusión es necesariamente verdadera.

Ejemplo:

> Regla: Todos los mamíferos son de sangre caliente.  
> Hecho: Los delfines son mamíferos.  
> Conclusión: Los delfines son de sangre caliente.

Un LLM alineado debería ser capaz de seguir este patrón de forma consistente.

#### **Razonamiento inductivo**

- De casos particulares -> regla general (hipótesis).  
- Las conclusiones son probables, no 100% seguras.

Ejemplo:

> Hemos visto que en varios experimentos, al aumentar el tamaño de los datos
> mejora el rendimiento del modelo.  
> Hipótesis: probablemente, más datos tiendan a mejorar el modelo en este rango de problemas.

#### **Razonamiento abductivo**

- De una observación -> mejor explicación posible.  
- Típico de diagnósticos (explicaciones plausibles, no ciertas).

Ejemplo:

> Observación: un sistema de clasificación que antes funcionaba bien ahora falla
> en datos recientes.  
> Explicaciones plausibles:  
> - Cambio de distribución de datos (data drift).  
> - Bug en el preprocesamiento de la nueva data.  
> - Nuevas clases o etiquetas mal definidas.

#### **Guiar al modelo con prompts**

Estrategias para fomentar buen razonamiento en un LLM:

- Pedir pasos explícitos:  
  - "Primero enumera tus premisas, luego razona, luego concluye".
- Asignar un rol:  
  - "Actúa como profesor de lógica", "actúa como auditor de seguridad".
- Combinar con auto-consistencia y recitación, especialmente en tareas complejas.


In [11]:
reasoning_prompt = (
    "Actúa como un profesor de lógica.\n"
    "Clasifica el siguiente fragmento de razonamiento como DEDUCTIVO, "
    "INDUCTIVO o ABDUCTIVO y explica por qué.\n\n"
    "Texto:\n"
    "\"En los últimos tres años, todos los modelos que entrenamos con más datos\n"
    "obtuvieron mejor rendimiento en validación. Por eso, creemos que entrenar "
    "con más datos ayudará a nuestro próximo modelo.\""
)

inputs = ppo_tokenizer(reasoning_prompt, return_tensors="pt").to(device)
outputs = policy_model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
)
gen = outputs[:, inputs["input_ids"].shape[-1]:]
print(ppo_tokenizer.batch_decode(gen, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




I hope you are interested in this project. It is not only the best thing to do. It is more important to me to take a look and learn how to todo.

I hope you will be happy with this project.

I hope you will have a great experience.

Thank you very much.

I hope that you will enjoy the project.

Thank you very much.

I hope that you will have a happy life.

Thank you very much.
Thank you very much.

I hope that you will have a happy life.
Thank you very much


#### **Ejemplo opcional con un modelo más grande (LLaMA 7B, conceptual)**

A modo de conexión mostramos ejemplo **conceptual**
(para hardware potente) usando un modelo tipo `huggyllama/llama-7b` y un argumento
`dola_layers='high'` (propio de una implementación concreta de DoLa).

> **Advertencia:** no ejecutes esto si no tienes GPU potente y el modelo descargado.
> Es principalmente ilustrativo.


In [12]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch
#
# MODEL_NAME = "huggyllama/llama-7b"
#
# tokenizer_llama = AutoTokenizer.from_pretrained(MODEL_NAME)
# model_llama = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )
#
# text = "Who shared a dorm with Harry Potter?"
# inputs = tokenizer_llama(text, return_tensors="pt").to(model_llama.device)
# output = model_llama.generate(
#     **inputs,
#     do_sample=False,
#     max_new_tokens=50,
#     # dola_layers='high',  # requerido por una implementación específica de DoLa
# )
# print(tokenizer_llama.batch_decode(
#     output[:, inputs["input_ids"].shape[-1]:],
#     skip_special_tokens=True
# ))